In [1]:
!pip install -q pdfplumber pandas scikit-learn sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 19.7 MB/s eta 0:00:00


In [2]:
import pdfplumber
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [3]:
CATEGORIES = {
    "FOOD": [
        "swiggy","zomato","eatsure","ubereats","restaurant","hotel","dhaba",
        "cafe","bakery","canteen","mess","fastfood","pizza","burger",
        "dominos","kfc","mcdonalds","subway","ccd","starbucks","barista",
        "sweets","icecream","dessert","catering"
    ],

    "FUEL": [
        "petrol","diesel","fuel","petroleum","pump","fuelstation","hpcl",
        "iocl","bpcl","indianoil","bharatpetroleum","shell","nayara",
        "cng","ev","charging","fastag","toll"
    ],

    "TELECOM": [
        "jio","airtel","vodafone","vi","bsnl","recharge","mobile",
        "postpaid","prepaid","fiber","broadband","internet",
        "dth","tataplay","dish"
    ],

    "EDUCATION": [
        "school","college","university","institute","academy",
        "education","fees","tuition","coaching","training",
        "byjus","unacademy","vedantu","physicswallah","pw",
        "exam","library","course","class"
    ],

    "GROCERY": [
        "grocery","supermarket","hypermarket","mart","kirana",
        "generalstore","provision","dmart","reliance","reliancefresh",
        "jiomart","bigbasket","grofers","blinkit","zepto",
        "bigbazaar","smartbazaar","spencers","dairy","milk",
        "fruits","vegetables"
    ],

    "MEDICAL": [
        "hospital","clinic","pharmacy","medical","chemist",
        "apollo","medplus","netmeds","pharmeasy","1mg",
        "tatamg","diagnostic","lab","pathology",
        "doctor","dental","eye","optical","health","wellness"
    ],

    "TRANSPORT": [
        "uber","ola","rapido","metro","bus","train","railway","irctc",
        "taxi","cab","auto","parking","toll","fastag"
    ],

    "TRAVEL": [
        "flight","airline","airport","pnr","airindia","indigo","vistara",
        "akasa","spicejet","makemytrip","goibibo","cleartrip","yatra"
    ],

    "SHOPPING": [
        "amazon","flipkart","myntra","ajio","meesho","nykaa","tatacliq",
        "store","shop","mall","retail","electronics","mobile",
        "laptop","appliance","fashion","clothing","footwear"
    ],

    "ENTERTAINMENT": [
        "movie","cinema","theatre","ticket","pvr","inox","bookmyshow",
        "netflix","primevideo","hotstar","spotify","gaana",
        "gaming","dream11","mpl","concert","event"
    ],

    "FINANCE": [
        "bank","upi","loan","emi","interest","investment","insurance",
        "lic","premium","tax","gst"
    ],

    "PERSONAL_TRANSFER": [
        "upi transfer","to friend","to family","salary","gift",
        "cash","deposit","withdraw"
    ],

    "RENT": ["rent","lease","landlord","accommodation"],
    "UTILITIES": ["electricity","water","gas","bill","bescom","indane"],
    "SUBSCRIPTION": ["subscription","autopay","recurring"],
    "BUSINESS": ["invoice","vendor","supplier","professional"],
    "GOVERNMENT": ["govt","government","passport","rto","license"],
    "CHARITY": ["donation","ngo","trust","fund"],
    "REAL_ESTATE": ["property","brokerage","registration"],
    "OTHER": []
}

ALL_CATEGORIES = list(CATEGORIES.keys())


In [4]:
pdf_path = "/content/Acct Statement_7529_07012026_14.37.54.csv.pdf"
pdf_password = "210745130"

rows = []

with pdfplumber.open(pdf_path, password=pdf_password) as pdf:
    for page in pdf.pages:
        text = page.extract_text(x_tolerance=1.5, y_tolerance=2)
        if not text:
            continue
        for line in text.split("\n"):
            line = re.sub(r"\s+", " ", line.strip())
            if re.match(r"\d{2}/\d{2}/\d{2}", line):
                rows.append(line)

raw_df = pd.DataFrame(rows, columns=["raw_line"])

pattern = re.compile(
    r"(?P<date>\d{2}/\d{2}/\d{2})\s+(?P<narration>.+?)\s+(?P<amount>\d[\d,]*\.\d{2})"
)

txns = []
for r in raw_df["raw_line"]:
    m = pattern.search(r)
    if m:
        d = m.groupdict()
        d["amount"] = float(d["amount"].replace(",", ""))
        txns.append(d)

df = pd.DataFrame(txns)
df["date"] = pd.to_datetime(df["date"], format="%d/%m/%y")


In [5]:
def extract_merchant(text):
    text = text.upper()
    text = re.sub(r"UPI[-/]", "", text)
    text = re.sub(r"@[\w]+", "", text)
    text = re.sub(r"\d{6,}", "", text)
    text = re.sub(r"[^A-Z ]", " ", text)
    return re.sub(r"\s+", " ", text).strip()

df["merchant_name"] = df["narration"].apply(extract_merchant)
df["text"] = (df["merchant_name"] + " " + df["narration"]).str.lower()


In [6]:
def rule_category(text):
    for cat, kws in CATEGORIES.items():
        for k in kws:
            if k in text:
                return cat, 0.95
    return None, None


In [7]:
train_samples = []
for cat, kws in CATEGORIES.items():
    for k in kws:
        train_samples.append((k, cat))

train_df = pd.DataFrame(train_samples, columns=["text","category"])

X_train, X_test, y_train, y_test = train_test_split(
    train_df["text"], train_df["category"], test_size=0.3, random_state=42
)

vectorizer = TfidfVectorizer(ngram_range=(1,2))
Xtr = vectorizer.fit_transform(X_train)
Xte = vectorizer.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(Xtr, y_train)

print("ML Accuracy:", accuracy_score(y_test, clf.predict(Xte)))
print(classification_report(y_test, clf.predict(Xte)))


ML Accuracy: 0.10144927536231885
                   precision    recall  f1-score   support

         BUSINESS       0.00      0.00      0.00         2
          CHARITY       0.00      0.00      0.00         2
        EDUCATION       0.00      0.00      0.00         7
    ENTERTAINMENT       0.00      0.00      0.00         5
          FINANCE       0.00      0.00      0.00         2
             FOOD       0.10      1.00      0.19         7
             FUEL       0.00      0.00      0.00         4
          GROCERY       0.00      0.00      0.00         7
          MEDICAL       0.00      0.00      0.00         8
PERSONAL_TRANSFER       0.00      0.00      0.00         5
             RENT       0.00      0.00      0.00         1
         SHOPPING       0.00      0.00      0.00         7
     SUBSCRIPTION       0.00      0.00      0.00         1
          TELECOM       0.00      0.00      0.00         2
        TRANSPORT       0.00      0.00      0.00         6
           TRAVEL     

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [8]:
final_cat = []
final_conf = []

for _, r in df.iterrows():
    cat, conf = rule_category(r["text"])
    if cat:
        final_cat.append(cat)
        final_conf.append(conf)
    else:
        prob = clf.predict_proba(vectorizer.transform([r["text"]]))[0]
        idx = np.argmax(prob)
        final_cat.append(clf.classes_[idx])
        final_conf.append(prob[idx])

df["final_category"] = final_cat
df["confidence"] = final_conf


In [9]:
category_summary = (
    df.groupby("final_category")
    .agg(
        number_of_transactions=("amount","count"),
        total_amount=("amount","sum")
    )
    .reset_index()
)

display(category_summary)


,final_category,number_of_transactions,total_amount
0,EDUCATION,2,3000.00
1,ENTERTAINMENT,4,3796.98
2,FINANCE,505,751914.99
3,FOOD,68,182467.29
4,FUEL,12,3040.00
5,GROCERY,6,1902.81
6,MEDICAL,13,4472.17
7,PERSONAL_TRANSFER,1,80000.00
8,SHOPPING,10,4533.00
9,TELECOM,66,695149.00


In [13]:
user_cat = input("Enter category name (e.g. FOOD, TELECOM, EDUCATION): ").upper()

details = df[df["final_category"] == user_cat]

if details.empty:
    print("No transactions found")
else:
    display(details[[
        "date","merchant_name","amount","confidence"
    ]])


Enter category name (e.g. FOOD, TELECOM, EDUCATION): finance


,date,merchant_name,amount,confidence
0,2025-04-01,PAYAL PRASHANT,45.0,0.95
1,2025-04-02,ANISHA DWIVEDI PPIW,80.0,0.95
2,2025-04-03,MR SHIVANG,9.0,0.95
3,2025-04-03,TUSHAR RAMKRUSHNA,10.0,0.95
4,2025-04-03,MR SANTOSHSINGH SON,5000.0,0.95
...,...,...,...,...
690,2026-01-02,CHAKKAPPANS,42.0,0.95
691,2026-01-02,CHAKKAPPANS,66.0,0.95
692,2026-01-04,RAMBALI,35.0,0.95
693,2026-01-06,OPENAI,1.0,0.95
